<a href="https://colab.research.google.com/github/LeenaGannu/Malaria-Detection-Using-DL/blob/main/ourcnncustom_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import os


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings(action="ignore")
from matplotlib.image import imread
from sklearn.metrics import classification_report
#tensorflow modules
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
data = "../input/cell-images-for-detecting-malaria/cell_images/cell_images/"


In [ ]:
os.listdir(data)


In [ ]:
uninfected = len(os.listdir("../input/cell-images-for-detecting-malaria/cell_images/cell_images/Uninfected"))
parasitized = len(os.listdir("../input/cell-images-for-detecting-malaria/cell_images/cell_images/Parasitized"))
print("Uninfected: ",uninfected)
print("Parasitized: ", parasitized)


In [ ]:
parasitized_data = os.listdir("../input/cell-images-for-detecting-malaria/cell_images/cell_images/Parasitized")


In [ ]:
sh=data+"/Parasitized"+"/C99P60ThinF_IMG_20150918_141001_cell_133.png"
print("Shape of image: ",imread(sh).shape)


In [ ]:
image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.1,
                               shear_range=0.1,
                               fill_mode="nearest")


In [ ]:
train = tf.keras.preprocessing.image_dataset_from_directory(data,
                                                            validation_split=0.2,
                                                            seed=123,
                                                            subset="training",
                                                            image_size=(134,131),
                                                            batch_size=32)
test = tf.keras.preprocessing.image_dataset_from_directory(data,
                                                           validation_split=0.2,
                                                           seed=123,
                                                           subset="validation",
                                                           image_size=(134,131),
                                                           batch_size=32)


In [ ]:
test.class_names

In [ ]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [ ]:
# defining sequential model
model = Sequential()

#adding convo-pool layers
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(134,131,3),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(134,131,3),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=128, kernel_size=(3,3),input_shape=(134,131,3),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))
model.add(Conv2D(filters=256, kernel_size=(3,3),input_shape=(134,131,3),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=2))

# flattening image
model.add(Flatten())

# adding dense layers
model.add(Dense(128,activation='relu'))
# adding dropout to minimize overfitting issue
model.add(Dropout(0.2))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

#compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=["accuracy"])

#summary of the model
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor="val_loss",patience=5, verbose=True)


In [ ]:
history = model.fit(train,validation_data=test,epochs=11,callbacks=[early_stop],verbose=True)


In [ ]:
import matplotlib.pyplot as plt
# plot the loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc'